In [1]:
from miditok import REMI, TokenizerConfig
import constants

tokenizer = REMI(params=constants.TOKENIZER_PATH)

/home/idoifergan/miniconda3/envs/cs236781-hw2/lib/python3.8/site-packages/miditok/midi_tokenizer.py:3252: UserWarning: The special token PAD_None is present twice in your configuration. Skipping its duplicated occurrence.
  self.config = TokenizerConfig()
/home/idoifergan/miniconda3/envs/cs236781-hw2/lib/python3.8/site-packages/miditok/classes.py:702: UserWarning: The special token PAD_None is present twice in your configuration. Skipping its duplicated occurrence.
  return cls(**input_dict, **kwargs)


In [2]:
print(len(tokenizer))
print(tokenizer.vocab_size)
print(tokenizer.config)
print(tokenizer.vocab)

30000
30000
{'PAD_None': 0, 'BOS_None': 1, 'EOS_None': 2, 'MASK_None': 3, 'Bar_None': 4, 'Pitch_21': 5, 'Pitch_22': 6, 'Pitch_23': 7, 'Pitch_24': 8, 'Pitch_25': 9, 'Pitch_26': 10, 'Pitch_27': 11, 'Pitch_28': 12, 'Pitch_29': 13, 'Pitch_30': 14, 'Pitch_31': 15, 'Pitch_32': 16, 'Pitch_33': 17, 'Pitch_34': 18, 'Pitch_35': 19, 'Pitch_36': 20, 'Pitch_37': 21, 'Pitch_38': 22, 'Pitch_39': 23, 'Pitch_40': 24, 'Pitch_41': 25, 'Pitch_42': 26, 'Pitch_43': 27, 'Pitch_44': 28, 'Pitch_45': 29, 'Pitch_46': 30, 'Pitch_47': 31, 'Pitch_48': 32, 'Pitch_49': 33, 'Pitch_50': 34, 'Pitch_51': 35, 'Pitch_52': 36, 'Pitch_53': 37, 'Pitch_54': 38, 'Pitch_55': 39, 'Pitch_56': 40, 'Pitch_57': 41, 'Pitch_58': 42, 'Pitch_59': 43, 'Pitch_60': 44, 'Pitch_61': 45, 'Pitch_62': 46, 'Pitch_63': 47, 'Pitch_64': 48, 'Pitch_65': 49, 'Pitch_66': 50, 'Pitch_67': 51, 'Pitch_68': 52, 'Pitch_69': 53, 'Pitch_70': 54, 'Pitch_71': 55, 'Pitch_72': 56, 'Pitch_73': 57, 'Pitch_74': 58, 'Pitch_75': 59, 'Pitch_76': 60, 'Pitch_77': 61, 'Pit

In [1]:
import torch
import torch.nn as nn
import constants
from miditok import REMI, TokenizerConfig
from tokenization_utils import get_dataloader
from torch.nn import TransformerDecoderLayer, TransformerDecoder
from model import MusicTransformer2, TransformerConfig


# Assume you have already initialized the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

tokenizer = REMI(params=constants.TOKENIZER_PATH)
config = TransformerConfig()
vocab_size = tokenizer.vocab_size
config.vocab_size = vocab_size
config.device = device

model = MusicTransformer2(config).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4)
dl_train = get_dataloader(constants.DATASET_TRAIN_PATH, constants.TOKENIZER_PATH)
print("Collected data loader")

cuda


/home/idoifergan/miniconda3/envs/cs236781-hw2/lib/python3.8/site-packages/miditok/midi_tokenizer.py:3252: UserWarning: The special token PAD_None is present twice in your configuration. Skipping its duplicated occurrence.
  self.config = TokenizerConfig()
/home/idoifergan/miniconda3/envs/cs236781-hw2/lib/python3.8/site-packages/miditok/classes.py:702: UserWarning: The special token PAD_None is present twice in your configuration. Skipping its duplicated occurrence.
  return cls(**input_dict, **kwargs)


Collected data loader


In [2]:
for batch in dl_train:
    x = batch["input_ids"].to(device)
    padding_mask = ~batch["attention_mask"].to(device, dtype=torch.bool)
    targets = torch.roll(x, shifts=-1, dims=1).to(device)
    targets[:, -1] = 0

    # Zero the gradients
    optimizer.zero_grad()
    
    logits = model(x, tgt_key_padding_mask=padding_mask)
    loss = criterion(logits.view(-1, vocab_size), targets.view(-1))
    loss.backward()

    optimizer.step()

    print(f'loss: {loss.item()}')

loss: 10.4375
loss: 10.0625
loss: 9.75
loss: 9.5625
loss: 9.4375
loss: 9.3125
loss: 9.25
loss: 9.125
loss: 9.0625
loss: 8.9375
loss: 8.875
loss: 8.8125
loss: 8.6875
loss: 8.625
loss: 8.5625
loss: 8.375
loss: 8.3125
loss: 8.125
loss: 8.25
loss: 8.1875
loss: 8.125
loss: 7.96875
loss: 7.96875
loss: 7.9375
loss: 7.875
loss: 7.75
loss: 8.0
loss: 7.65625
loss: 7.84375
loss: 7.84375
loss: 7.90625
loss: 7.71875
loss: 7.9375
loss: 7.96875
loss: 7.875
loss: 7.8125
loss: 7.78125
loss: 7.75
loss: 7.6875
loss: 7.625
loss: 7.875
loss: 7.6875
loss: 7.71875
loss: 7.8125
loss: 7.59375
loss: 7.71875
loss: 7.6875
loss: 7.78125
loss: 7.96875
loss: 7.5
loss: 7.5625
loss: 7.6875
loss: 7.8125
loss: 7.71875
loss: 7.75
loss: 7.65625
loss: 7.8125
loss: 7.625
loss: 7.6875
loss: 7.65625
loss: 7.8125
loss: 7.65625
loss: 7.6875
loss: 7.625
loss: 7.375
loss: 7.5
loss: 7.59375
loss: 7.71875
loss: 7.5625
loss: 7.78125
loss: 7.5625
loss: 7.5
loss: 7.5
loss: 7.6875
loss: 7.59375
loss: 7.625
loss: 7.46875
loss: 7.6875
lo